In [14]:
#Q1: Does patient gender predict APACHE score? (SE)
#Q1A: Does patient gender predict APACHE IV score? (SE)
#Q1B: Does patient gender predict APACHE IVa score? (SE)

import pandas as pd
import numpy as np
import csv
import scipy as sp
import scipy.stats as stat

In [15]:
pts = pd.read_csv('patient.csv')

In [16]:
apachescores = pd.read_csv('apachePatientResult.csv')

In [17]:
apachescores.head()

,apachepatientresultsid,patientunitstayid,physicianspeciality,physicianinterventioncategory,acutephysiologyscore,apachescore,apacheversion,predictedicumortality,actualicumortality,predictediculos,...,predictedhospitallos,actualhospitallos,preopmi,preopcardiaccath,ptcawithin24h,unabridgedunitlos,unabridgedhosplos,actualventdays,predventdays,unabridgedactualventdays
0,26570,141168,critical care medicine (CCM),Unknown,49,65,IV,0.026988,EXPIRED,3.038388,...,7.546453,2.4972,0,0,0,2.4972,2.4972,NaN,NaN,NaN
1,26571,141168,critical care medicine (CCM),Unknown,49,65,IVa,0.028889,EXPIRED,3.091127,...,6.628720,2.4972,0,0,0,2.4972,2.4972,NaN,NaN,NaN
2,53135,141194,critical care medicine (CCM),Unknown,57,70,IV,0.037888,ALIVE,4.620982,...,13.338449,9.2167,0,0,0,3.3423,9.2167,NaN,NaN,NaN
3,53136,141194,critical care medicine (CCM),Unknown,57,70,IVa,0.046448,ALIVE,4.167129,...,12.978228,9.2167,0,0,0,3.3423,9.2167,NaN,NaN,NaN
4,8,141203,hospitalist,I,73,90,IVa,0.291609,ALIVE,8.670299,...,16.319389,3.7493,0,0,0,1.2979,3.7493,2.0,5.738093,2.0


In [18]:
# in case per patient analysis needed
unipts = pts.drop_duplicates('uniquepid')

In [19]:
# for now continuing with analysis per trauma admission (i.e. if patient X comes in twice for trauma, n = 2 for that patient)

q = 'trauma'

mpts = pts[pts['gender'] == 'Male']
fpts = pts[pts['gender'] == 'Female']

mepts = mpts[mpts['unitadmitsource'] == 'Emergency Department']
fepts = fpts[fpts['unitadmitsource'] == 'Emergency Department']

# male and female trauma ED-ICU admissions
msepts = mepts[mepts['apacheadmissiondx'].str.contains(q).fillna(False)]
fsepts = fepts[fepts['apacheadmissiondx'].str.contains(q).fillna(False)]

In [20]:
msepts.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
14,141264,128998,Male,19,Caucasian,59,91,Head only trauma,188.0,02:24:29,...,Emergency Department,1,admit,NaN,80.1,09:02:00,292,Step-Down Unit (SDU),Alive,002-47110
488,144519,131512,Male,56,Caucasian,68,103,Head only trauma,180.3,23:07:00,...,Emergency Department,1,admit,99.80,105.4,20:42:00,1293,Floor,Alive,002-33534
511,144689,131641,Male,57,Caucasian,73,97,Extremity only trauma,170.2,23:15:00,...,Emergency Department,1,admit,49.89,55.6,21:06:00,4190,Floor,Alive,002-38176
665,145604,132349,Male,85,Hispanic,73,100,Head only trauma,152.4,14:48:00,...,Emergency Department,1,admit,63.50,54.0,04:43:00,2273,Floor,Alive,002-46870
747,146127,132760,Male,51,African American,73,99,Head/face trauma,177.8,19:31:00,...,Emergency Department,1,admit,79.30,73.8,21:39:00,1559,Step-Down Unit (SDU),Alive,002-44708


In [62]:
mreind = msepts.reset_index()
freind = fsepts.reset_index()


ma4 = []
fa4 = []
ma4a = []
fa4a = []

for i in range(50):
    for j in range(len(apachescores)):
        if mreind['patientunitstayid'][i] == apachescores['patientunitstayid'][j]:
            if apachescores['apacheversion'][j] == 'IV':
                ma4.append(apachescores['apachescore'][j])
            if apachescores['apacheversion'][j] == 'IVa':
                ma4a.append(apachescores['apachescore'][j])
                

for i in range(50):
    for j in range(len(apachescores)):
        if freind['patientunitstayid'][i] == apachescores['patientunitstayid'][j]:
            if apachescores['apacheversion'][j] == 'IV':
                fa4.append(apachescores['apachescore'][j])
            if apachescores['apacheversion'][j] == 'IVa':
                fa4a.append(apachescores['apachescore'][j])

# Q tech, is there a way to do this pythonically with indexing rather than for loops?

In [ ]:
for i in range(len(mreind)):
    for j in range(len(apachescores)):
        if mreind['patientunitstayid'][i] == apachescores['patientunitstayid'][j]:
            if apachescores['apacheversion'][j] == 'IV':
                ma4.append(apachescores['apachescore'][j])
            if apachescores['apacheversion'][j] == 'IVa':
                ma4a.append(apachescores['apachescore'][j])

for i in range(len(freind)):
    for j in range(len(apachescores)):
        if freind['patientunitstayid'][i] == apachescores['patientunitstayid'][j]:
            if apachescores['apacheversion'][j] == 'IV':
                fa4.append(apachescores['apachescore'][j])
            if apachescores['apacheversion'][j] == 'IVa':
                fa4a.append(apachescores['apachescore'][j])

In [63]:
print np.mean(ma4a)
print np.mean(fa4a)

39.28888888888889
41.644444444444446


In [ ]:
msepts.insert(1,'Apache IV', ma4)
fsepts.insert(1,'Apache IV', fa4)
msepts.insert(2,'Apache IVa', ma4a)
fsepts.insert(2,'Apache IVa', fa4a)


In [66]:
print stat.ttest_ind(ma4, fa4)

print stat.ttest_ind(ma4a, fa4a)

Ttest_indResult(statistic=-0.558815479313645, pvalue=0.5777069308324319)
Ttest_indResult(statistic=-0.558815479313645, pvalue=0.5777069308324319)
